# Script to get values plotted in Figure 1E

Calculates reads normalized to feature length and then normalized again to average CDS coverage for 5'UTRs, 3'UTRs, and introns. Only features that were unique and non-overlapping were included. The corresponding CDS needed to have coverage of at least 128 reads for any feature to be counted. 

In [17]:
import glob, pathlib

#dictionaries to store feature counts for each sample
five_prime_utr = {}
three_prime_utr = {}
intron = {}

unique_transcripts = set()
with open("unique_features.gff", "r") as unique_features:
    for line in unique_features:
        data = line.strip().split("\t")
        transcript = data[8].split("transcript_id ")[1].replace('"', "")
        if (data[2] == "CDS"):
            unique_transcripts.add(transcript)
        if (data[2] == "five_prime_utr"):
            five_prime_utr[transcript] = {}
        if (data[2] == "three_prime_utr"):
            three_prime_utr[transcript] = {}
        if (data[2] == "intron"):
            intron[transcript] = {}
        
samples = ["rna_wt_25_1", "rna_wt_25_2", "rna_wt_25_3", 
           "rna_wt_37_1", "rna_wt_37_2", "rna_wt_37_3",
           "rna_ts_25_1", "rna_ts_25_2", "rna_ts_25_3",
           "rna_ts_37_1", "rna_ts_37_2", "rna_ts_37_3",
           "rpf_wt_25_1", "rpf_wt_25_2",
           "rpf_wt_37_1", "rpf_wt_37_2",
           "rpf_ts_25_1", "rpf_ts_25_2",
           "rpf_ts_37_1", "rpf_ts_37_2"]

feature_types = ["five_prime_utr", "CDS", "three_prime_utr", "intron"]

for sample in samples:
    #make dictionaries to store unique transcript information
    #element 1 = 5'UTR, 2 = CDS, 3 = 3'UTR, and 4 = intron; same as feature_types above
    counts = {}
    lengths = {}
    for unique_transcript in unique_transcripts:
        counts[unique_transcript] = [0, 0, 0, 0]
        lengths[unique_transcript] = [0, 0, 0, 0]
    
    #now go through all the FeatureCounts files to get read counts for each feature
    for feature_type in feature_types:
        #get the index of the feature type for proper assignment in the counts and lengths dictionaries
        feature_index = feature_types.index(feature_type)
        #open the corresponding FeatureCounts file
        with open(sample + "_" + feature_type + ".txt", "r") as in_file:
            for line in in_file:
                #skip the first two lines of the file
                if ("#" not in line and "Geneid" not in line):
                    #grab the transcript id (first column)
                    data = line.strip().split("\t")
                    transcript_id = data[0]
                    #grab the feature length (6th column) and read count (7th column)
                    #assign to their respective dictionary
                    feature_length = int(data[5])
                    reads = int(data[6])
                    lengths[transcript_id][feature_index] = feature_length
                    counts[transcript_id][feature_index] = reads

    #Once all features are parsed, calculate statistics for features
    for unique_transcript in unique_transcripts:
        read_counts = counts[unique_transcript]
        feature_lengths = lengths[unique_transcript]
        cds_counts = read_counts[1]
        cds_length = feature_lengths[1]
        
        if (cds_counts > 0):
            cds_average = cds_counts / cds_length        
            
            #store data for 5'UTR if there are read counts
            five_prime_utr_counts = read_counts[0]
            five_prime_utr_length = feature_lengths[0]
            if (five_prime_utr_length > 0 and five_prime_utr_counts + cds_counts >= 128):
                five_prime_utr_average = five_prime_utr_counts / five_prime_utr_length
                five_prime_utr[unique_transcript][sample] = five_prime_utr_average / cds_average
            
            #Same for 3'UTRs
            three_prime_utr_counts = read_counts[2]
            three_prime_utr_length = feature_lengths[2]
            if (three_prime_utr_length > 0 and three_prime_utr_counts + cds_counts >= 128):
                three_prime_utr_average = three_prime_utr_counts / three_prime_utr_length
                three_prime_utr[unique_transcript][sample] = three_prime_utr_average / cds_average
            
            #Same for introns
            intron_counts = read_counts[3]
            intron_length = feature_lengths[3]
            if (intron_length > 0 and intron_counts + cds_counts >= 128):
                intron_average = intron_counts / intron_length
                intron[unique_transcript][sample] = intron_average / cds_average

with open("five_prime_utr_counts.txt", "w") as out_file:    
    #write out samples as header line
    out_file.write("transcript\t")
    out_file.write("\t".join(samples))
    out_file.write("\n")
    
    #write out values for unique transcripts for each sample  
    for transcript, normalized_counts in five_prime_utr.items():
        out_file.write(transcript + "\t")
        out_file.write("\t".join(str(value) for value in normalized_counts.values()))
        out_file.write("\n")
        
with open("three_prime_utr_counts.txt", "w") as out_file:
    #write out samples as header line
    out_file.write("transcript\t")
    out_file.write("\t".join(samples))
    out_file.write("\n")
    
    #write out values for unique transcripts for each sample  
    for transcript, normalized_counts in three_prime_utr.items():
        out_file.write(transcript + "\t")
        out_file.write("\t".join(str(value) for value in normalized_counts.values()))
        out_file.write("\n")
        
with open("intron_counts.txt", "w") as out_file:
    #write out samples as header line
    out_file.write("transcript\t")
    out_file.write("\t".join(samples))
    out_file.write("\n")
    
    #write out values for unique transcripts for each sample  
    for transcript, normalized_counts in intron.items():
        out_file.write(transcript + "\t")
        out_file.write("\t".join(str(value) for value in normalized_counts.values()))
        out_file.write("\n")

# Script to get pseudo A-site coordinates from genome mapped ribosome profiling reads

Uses the bed converted bam file start and end coordinates. Finds the midpoint of those two coordinates and biases towards the 5' end if read length is odd (eg. for a + strand read, the A-site is calculated as 16 if the midpoint is 16.5). 

In [7]:
import glob, math

#Mapped ribosome profiling reads converted from bam to bed files so we get their 5 and 3' ends
for file in glob.glob("rpf_*.bed"):
    with open(file, "r") as in_file, open(file.replace(".bed", "_coverage.bed"), "w") as out_file:
        for line in in_file:
            data = line.strip().split("\t")
            #write out the chromosome value to the output file
            out_file.write(data[0] + "\t")
            #start coordinate
            start = int(data[1])
            #end coordinate
            end = int(data[2])
            #Pseudo A-site is the mid point with bias towards the 5' end of the gene
            #data[5] is the strand value. If + strand, then bias towards the lower coordinate (the 5' end)
            #If - strand, then bias towards the higher coordinate (the 5' end)            
            if (data[5] == "+"):
                start = math.floor((start + end) / 2)
            else:
                start = math.ceil((start + end) / 2)
            #We only care about the A-site coordinate so the end and start coordinate are the same    
            end = start
            #write out the coordinates for the A-site
            out_file.write(str(start) + "\t" + str(end) + "\t")
            #copy the rest of the values in the bed file to the output file
            out_file.write(data[3] + "\t" + data[4] + "\t" + data[5] + "\n")

# Script to get values plotted in Figure 1F

Calculates metagene profiles. Uses coverage from the entire RNA-seq read (Bedtools) or the pseudo A-site for ribosome profiling reads (calculated above), for all unique features. Requires CDS to have at least 128 reads of coverage and for the 5'UTR, CDS, and 3'UTR to be at least 100 nt in length each. Coverage at each position is normalized to average coverage across the whole gene. Coverage at each position in each feature (5'UTR, CDS, 3'UTR) is divided across 100 bins and then averaged across all features of the same type using the SciPy library. 

In [25]:
import glob, pathlib, numpy, collections
from scipy import stats

featureTypes = ["five_prime_utr", "CDS", "three_prime_utr"]

#Coverages are in bed files
for file in glob.glob("*_coverage.bed"):
    #output metagene coverages for each input file
    with open(file, "r") as in_file, open(file.replace("_coverage.bed", "_metagene.txt"), "w") as out_file:
        transcripts = {}
        for line in in_file:
            data = line.strip().split("\t")
            featureType = data[2]
            if (featureType in featureTypes):
                transcript = data[8].split("transcript_id ")[1].replace('"', "")
                
                #Create new entries for transcripts if they haven't already
                #Entry is [[5'UTR], [CDS], [3'UTR]]
                if (transcript not in transcripts):
                    transcripts[transcript] = [collections.deque(), collections.deque(), collections.deque()]

                featureIndex = featureTypes.index(featureType)        

                strand = data[6]
                reads = int(data[10])
                
                #If strand is positive, add elements to the end to maintain 5' to 3' orientation
                #If not, add elements to the beginning to maintain 5' to 3' orientation
                if (strand == "+"):
                    transcripts[transcript][featureIndex].append(reads)
                else:
                    transcripts[transcript][featureIndex].appendleft(reads)
        
        meta_five_prime_utr = collections.deque()
        meta_cds = collections.deque()
        meta_three_prime_utr = collections.deque()

        for transcript, counts in transcripts.items():
            five_prime_utr = counts[0]
            cds = counts[1]
            three_prime_utr = counts[2]

            #Only calculate for transcripts with at least 128 reads of coverage and 100 nt in length
            #Also make sure 5' and 3'UTR are at least 100 nt in length
            if (sum(cds) >= 128 and len(cds) >= 100):
                if (len(five_prime_utr) >= 100 and len(three_prime_utr) >= 100):
                    total_coverage = numpy.array(five_prime_utr + cds + three_prime_utr)
                    mean_coverage = numpy.mean(total_coverage)

                    five_prime_utr = numpy.array(five_prime_utr)
                    cds = numpy.array(cds)
                    three_prime_utr = numpy.array(three_prime_utr)
                    
                    #Divide coverages across 100 equal bins -- our "metagene"
                    cds_binned = stats.binned_statistic(range(0, len(cds)), cds / mean_coverage, "mean", bins = 100)
                    five_prime_utr_binned = stats.binned_statistic(range(0, len(five_prime_utr)), 
                                                                   five_prime_utr / mean_coverage, "mean", bins = 100)
                    three_prime_utr_binned = stats.binned_statistic(range(0, len(three_prime_utr)), 
                                                                    three_prime_utr / mean_coverage, "mean", bins = 100)

                    meta_five_prime_utr.append(five_prime_utr_binned[0])
                    meta_cds.append(cds_binned[0])
                    meta_three_prime_utr.append(three_prime_utr_binned[0])

        meta_five_prime_utr = numpy.array(meta_five_prime_utr)
        meta_cds = numpy.array(meta_cds)
        meta_three_prime_utr = numpy.array(meta_three_prime_utr)
        
        #Average coverage at each position in each feature across all transcripts
        meta_five_prime_utr = numpy.mean(meta_five_prime_utr, axis=0)                                                
        meta_cds = numpy.mean(meta_cds, axis=0)                                                 
        meta_three_prime_utr = numpy.mean(meta_three_prime_utr, axis=0)

        #Write out values: first 100 are 5'UTR, next 100 are CDS, last 100 are 3'UTR
        for value in meta_five_prime_utr:
            out_file.write(str(value) + "\n")
        for value in meta_cds:
            out_file.write(str(value) + "\n")
        for value in meta_three_prime_utr:
            out_file.write(str(value) + "\n")